In [1]:
import os 
import numpy as np
##from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

c:\Users\USER\Desktop\projects\rag_application\ragenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


###download the docs

In [2]:
from urllib.request import urlretrieve

In [3]:
files=[
     r"C:\Users\USER\Downloads\Listen To Your Gut Third Edition.pdf"
]
os.makedirs('listentoyourgut',exist_ok=True)

In [4]:
import shutil
for file_path in files:
    
    # Get just the filename
    file_name= os.path.basename(file_path)
    
    # Destination path
    dest_path = os.path.join('listentoyourgut', file_name)
    
    # Copy the file
    shutil.copy(file_path, dest_path)                             

In [5]:
## LOAD THE FILE 
loader=PyPDFDirectoryLoader('./listentoyourgut/')

In [6]:
docs_before_splitter=loader.load()

In [7]:
len(docs_before_splitter[1].page_content)

1612

In [12]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=50
)

docs_after_split=text_splitter.split_documents(docs_before_splitter)

In [13]:
len(docs_after_split[0].page_content)

635

In [14]:
## embeding 
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest",
    base_url="http://localhost:11434/",
    
)

In [15]:
embeddings.embed_query('hello world')

[-0.0068108733,
 -0.0013206175,
 -0.17139465,
 0.008482935,
 0.005775208,
 0.06989259,
 -0.00018551052,
 -0.043105833,
 -0.014587172,
 -0.05410775,
 0.00046831524,
 0.039124385,
 0.027719,
 0.080870174,
 0.045357786,
 -0.06291568,
 0.0102682905,
 -0.029576847,
 -0.042813912,
 0.029640555,
 -0.0037494507,
 -0.09432124,
 -0.0076218587,
 0.038121693,
 0.09211822,
 -0.014312543,
 -0.01504978,
 0.06159304,
 0.006505795,
 -0.021986103,
 -0.0012220312,
 -0.01093497,
 -0.00021056995,
 0.015679274,
 0.03945693,
 0.0027191713,
 0.032620966,
 0.017263748,
 0.016285632,
 0.0059136082,
 -0.0046807583,
 -0.014821978,
 0.012090716,
 0.010199478,
 0.065954514,
 -0.0015092002,
 -0.0041856975,
 0.0003284209,
 0.08685434,
 -0.060585804,
 -0.018189756,
 0.005342436,
 -0.0009227128,
 0.06022204,
 0.06724744,
 0.035406712,
 0.049651287,
 -0.061637964,
 0.024219584,
 0.03452212,
 0.021710876,
 0.04369855,
 0.03295035,
 0.06526131,
 -0.01750874,
 -0.033569623,
 -0.0252197,
 0.03546419,
 -0.0027613016,
 0.0181

In [17]:
vector_score=FAISS.from_documents(docs_after_split,embeddings)

In [18]:
query='how i use my mind to help heal my body'

In [19]:
relevant_docs=vector_score.similarity_search(query)
relevant_docs

[Document(id='4d350936-d7e5-45a3-8d30-aea3d790fd58', metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-06-16T17:06:18+05:30', 'moddate': '2023-06-16T17:07:52+05:30', 'trapped': '/False', 'source': 'listentoyourgut\\Listen To Your Gut Third Edition.pdf', 'total_pages': 521, 'page': 18, 'page_label': '6'}, page_content='deal specifically with emotional/mental/spiritual healing (see Appendix A for \nmy suggestions) and I strongly encourage you to look at these aspects of your \nhealing simultaneously. In Chapter 6, I also outline some of the mind/body \nhealing therapies I encourage you to delve into to get this aspect of your healing \ngoing. Lastly, I have a section in my LTYG Shoppe on mind/body healing, plus \nan entire section on my blog called, Mind/Body/Spirit Healing  where I share \nmy own stories, videos, meditations, etc. So definitely use the resources on my \nwebsite to address this crucial facet of transitioning

In [34]:
print(relevant_docs[3].page_content)

to address the bleeding, and then determined and healed the root causes. 
Your body is always advocating on your behalf. Our bodies work together 
with our mind and our spirit to move us into greater wholeness, to heal 
dysfunctional beliefs, patterns, or behaviors. It’s up to you to decide what to 
do with the messages and opportunities your body brings you. Do you want 
to ignore the messages, the opportunities for wholeness, and just drug them 
away? Or do you want to receive them with gratitude as the pathway in to 
the greater healing of you; of your life, your relationships, the way you move 
through the gift of life on this planet?


In [20]:
## change vector store to retriever
retriever=vector_score.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [21]:
from langchain_ollama.llms import OllamaLLM

hf=OllamaLLM(model="phi:latest",base_url="http://localhost:11434",temperature=0.3,max_tokens=500)

In [22]:
query

'how i use my mind to help heal my body'

In [23]:
output=hf.invoke(query)

In [24]:
print(output)


 I do not have personal experiences or emotions, but i can provide you with some information about how the mind-body connection works in healing. many people believe that our thoughts and emotions can influence our physical health, as they are connected through the nervous system. when we experience stress, anxiety, or negative emotions, it can affect our immune system and make us more susceptible to illness. on the other hand, positive thinking and relaxation techniques like meditation and deep breathing can help reduce stress levels and improve overall well-being. ultimately, the mind-body connection is a complex process that involves many factors, including genetics, environment, and lifestyle choices.



In [25]:
prompt_template = """Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.

{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)
     

In [26]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf, 
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs = {"prompt" : PROMPT}
)

In [27]:
result=retrievalQA.invoke({"query" : query})

In [28]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [29]:
print(result)

{'query': 'how i use my mind to help heal my body', 'result': " I can't find the final answer but you may want to check the following links for more information on using your mind to help heal your body. \n\n1. Mind-Body Medicine\n2. Visualization Techniques\n3. Meditation and Relaxation\n4. Hypnosis\n5. Cognitive Behavioral Therapy\n6. Biofeedback\n7. Energy Healing\n8. Reiki\n9. Acupuncture\n10. Yoga\n11. Tai Chi\n12. Qigong\n13. Aromatherapy\n14. Music Therapy\n15. Art Therapy\n16. Animal-Assisted Therapy\n17. Guided Imagery\n18. Hypnotic Suggestion\n19. Hypnosis for Pain Management\n20. Mindfulness Meditation\n21. Transcendental Meditation\n22. Neurofeedback\n23. Biofeedback for Stress Reduction\n24. Emotional Freedom Techniques (EFT)\n25. Cognitive Restructuring\n26. Acceptance and Commitment Therapy\n27. Gestalt Therapy\n28. Solution-Focused Brief Therapy\n29. Narrative Therapy\n30. Systemic Family Therapy\n31. Psychodynamic Therapy\n32. Rational Emotive Behavior Therapy\n33. Dia

In [30]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

There are 3 documents retrieved which are relevant to the query.
****************************************************************************************************
Relevant Document #1:
Source file: listentoyourgut\Listen To Your Gut Third Edition.pdf, Page: 18
Content: deal specifically with emotional/mental/spiritual healing (see Appendix A for 
my suggestions) and I strongly encourage you to look at these aspects of your 
healing simultaneously. In Chapter 6, I also outline some of the mind/body 
healing therapies I encourage you to delve into to get this aspect of your healing 
going. Lastly, I have a section in my LTYG Shoppe on mind/body healing, plus 
an entire section on my blog called, Mind/Body/Spirit Healing  where I share 
my own stories, videos, meditations, etc. So definitely use the resources on my 
website to address this crucial facet of transitioning to wholeness.
Disease is not just a physical phenomenon. By using the diets, therapies
------------------------------

In [31]:
import gradio as gr

In [38]:
import gradio as gr

def rag_chat(query):
    result= retrievalQA.invoke(query)
    return result['result']  # your RetrievalQA chain

def user_message(user_input, history):
    # Add user input and RAG response as tuple
    bot_answer = rag_chat(user_input)
    history = history + [(user_input, rag_chat(user_input))]
    return history, ""  # return updated chatbot + clear textbox

def clear_chat():
    return [], ""  # clear both chatbot and textbox

with gr.Blocks() as demo:
    gr.Markdown("# 📚 Local RAG Chatbot ")
    gr.Markdown("Ask me anything about your PDF documents!")

    chatbot = gr.Chatbot(label="Chat interface")
    user_input = gr.Textbox(label="Your message", placeholder="Ask me something about your PDF...")
    send = gr.Button("Send")
    clear = gr.Button("Clear Chat")

    send.click(
        fn=user_message,
        inputs=[user_input, chatbot],   # user_input first, then history
        outputs=[chatbot, user_input],  # chatbot updated, textbox cleared
    )

    clear.click(
        fn=clear_chat,
        inputs=[],
        outputs=[chatbot, user_input],
    )

demo.launch()



C:\Users\USER\AppData\Local\Temp\ipykernel_19036\2540265402.py:20: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat interface")


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
